In [1]:
#!pip install tensorflow_hub
#!pip install bert-tensorflow
import pandas as pd
import numpy as np
import pickle
import tensorflow as tf
import tensorflow_hub as hub
import bert
from bert import tokenization
from IPython.display import display

## IG

In [2]:
from bert_model_utils import transform_input, generate_baseline, get_ig_attributions, visualize_token_attrs, get_tensor

In [3]:
# Initialize a Tensorflow Session and load the saved model
SESS = tf.Session()
SAVED_MODEL_PATH = 'bert_saved_model_sep19/1568924123'
saved_model = tf.saved_model.loader.load(sess=SESS, tags=['serve'], export_dir=SAVED_MODEL_PATH)


W1111 03:29:29.183141 139904582027072 deprecation.py:323] From <ipython-input-3-1491798eed86>:4: load (from tensorflow.python.saved_model.loader_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
W1111 03:29:31.317682 139904582027072 deprecation.py:323] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/training/saver.py:1282: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


In [4]:
# Define the input tensors using the SignatureDef
INPUT_TENSORS = saved_model.signature_def["serving_default"].inputs

# Get output tensor for model using the SignatureDef
OUTPUT_TENSOR = get_tensor(SESS, saved_model.signature_def["serving_default"].outputs['probabilities'].name)

# Path to Tokenizer
TOKENIZER_PATH = 'bert_tokenizer.pkl'
with open(TOKENIZER_PATH, 'rb') as file:
    TOKENIZER = pickle.load(file)
    
# The embedding tensor of the model
EMBEDDING_TENSOR = get_tensor(SESS, 'module_apply_tokens/bert/embeddings/add_1:0')

# Gradient tensor of output wrt embedding tensor
GRADIENT_TENSOR = tf.gradients(OUTPUT_TENSOR[:, 1], EMBEDDING_TENSOR)
    


W1111 03:29:33.593637 139904582027072 deprecation.py:323] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1205: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [5]:
# Create a DataFrame structure that we will input into the model
sentences = ['This was an awful movie', 'This was a good movie', 'This was not a good movie', 
             'This was not a great movie, but a good movie nevertheless',
            'This was a terrible movie. Do you agree?']

input_df = pd.DataFrame(columns=['sentence'], data=sentences)



for sentence in sentences:
    input_df['sentence'][0] = sentence
    transformed_input_df = transform_input(TOKENIZER, input_df.head(1))
    baseline_df = generate_baseline(TOKENIZER, input_df.head(1))
    ig = get_ig_attributions(sess=SESS, 
                             input_tensors=INPUT_TENSORS, 
                             embedding_tensor=EMBEDDING_TENSOR,
                             gradient_tensor=GRADIENT_TENSOR, 
                             output_tensor=OUTPUT_TENSOR, 
                             transformed_input_df=transformed_input_df,
                             baseline_df=baseline_df, 
                             tokenizer=TOKENIZER)
    display(visualize_token_attrs(ig['outputs'][0], np.array(ig['outputs'][1])))

prediction is 1.955999141500797e-05
baseline_prediction is 0.4553076922893524
delta_prediction is -0.4552881419658661
sum_attributions are -0.4555169343948364
Error percentage is -0.050252226641011435


prediction is 0.9998936653137207
baseline_prediction is 0.4553076922893524
delta_prediction is 0.5445859432220459
sum_attributions are 0.568222165107727
Error percentage is -4.340218872679179


prediction is 8.560651622246951e-05
baseline_prediction is 0.49438032507896423
delta_prediction is -0.4942947328090668
sum_attributions are -0.4316096305847168
Error percentage is 12.68172571212965
Num reps is 6, abs error percentage is 12.68172571212965
prediction is 8.560651622246951e-05
baseline_prediction is 0.49438032507896423
delta_prediction is -0.4942947328090668
sum_attributions are -0.4943336844444275
Error percentage is -0.007880244877252977


prediction is 0.9999759197235107
baseline_prediction is 0.5354936718940735
delta_prediction is 0.46448224782943726
sum_attributions are 1.131194829940796
Error percentage is -143.53887263226096
Num reps is 6, abs error percentage is -143.53887263226096
prediction is 0.9999759197235107
baseline_prediction is 0.5354936718940735
delta_prediction is 0.46448224782943726
sum_attributions are 0.3986581265926361
Error percentage is 14.171504195133945
Num reps is 11, abs error percentage is 14.171504195133945
prediction is 0.9999759197235107
baseline_prediction is 0.5354936718940735
delta_prediction is 0.46448224782943726
sum_attributions are 0.4558603763580322
Error percentage is 1.856232721852283


prediction is 1.669874654908199e-05
baseline_prediction is 0.5070011615753174
delta_prediction is -0.5069844722747803
sum_attributions are -0.5019686818122864
Error percentage is 0.9893380836672628
